In [3]:
!wget https://storage.googleapis.com/a-chill-bucket/enfusion_single_note_large_epoch%3D4999-step%3D110000.ckpt

--2023-01-20 11:05:40--  https://storage.googleapis.com/a-chill-bucket/enfusion_single_note_large_epoch%3D4999-step%3D110000.ckpt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.181.240, 142.250.185.144, 142.250.74.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.181.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 263692701 (251M) [application/octet-stream]
Saving to: ‘enfusion_single_note_large_epoch=4999-step=110000.ckpt’

enfusion_single_not 100%[===================>] 251.48M  89.4MB/s    in 2.8s    

2023-01-20 11:05:43 (89.4 MB/s) - ‘enfusion_single_note_large_epoch=4999-step=110000.ckpt’ saved [263692701/263692701]



In [11]:
from text2sound import text2sound, plot_and_hear

audio = text2sound("monsterous bass", 100, 4)

plot_and_hear(audio, 48000)

  0%|          | 0/100 [00:00<?, ?it/s]